# 📈 Manpan 情報網 v4.1 - 台股完整版

## 🎯 六大守護者系統

### 守護者功能說明
1. **守護者 1：市場熔斷** - 判斷大盤是否安全（加權指數 vs 季線）
2. **守護者 2：流動性** - 過濾成交金額太低的股票
3. **守護者 3：籌碼面** - 檢查三大法人（外資、投信、自營商）
4. **守護者 4：技術面** - 檢查股價是否過熱（乖離率）
5. **守護者 5：出場策略** - 停損停利機制
6. **守護者 6：倉位配置** - 根據評分決定買多少

---

## 📊 第一步：設定參數（可以自己調整）

In [ ]:
# ==================== 📌 參數設定區 ====================
# 這些數字都可以自己調整！

CONFIG = {
    # 守護者 1：市場熔斷
    "MARKET_MA60_PERIOD": 60,           # 季線（60 個交易日）
    "MARKET_LIMIT_DOWN_THRESHOLD": 100, # 跌停股票超過 100 支就危險
    
    # 守護者 2：流動性（台股特色：用成交金額判斷）
    "MIN_TURNOVER": 50_000_000,         # 最少日均成交 5000 萬台幣
    "VOLUME_SPIKE_RATIO": 5,            # 爆量警示：今日量 > 5 倍平均
    
    # 守護者 3：籌碼面（三大法人）
    "FOREIGN_BUY_RATIO": 0.05,          # 外資買超佔比 > 5%
    "TRUST_BUY_RATIO": 0.03,            # 投信買超佔比 > 3%
    "CONSECUTIVE_BUY_DAYS": 3,          # 連續買超 3 天
    
    # 守護者 4：技術面（乖離率）
    "BIAS_THRESHOLD_BULL": 0.30,        # 多頭市場：允許乖離 30%
    "BIAS_THRESHOLD_BEAR": 0.15,        # 空頭市場：只允許 15%
    
    # 守護者 5：出場策略
    "STOP_LOSS": 0.08,                  # 停損：-8%
    "TRAILING_STOP": 0.10,              # 移動停利：從高點回落 10%
    "TAKE_PROFIT": 0.30,                # 獲利了結：+30%
    "HOLDING_DAYS_MIN": 3,              # 持有至少 3 天才啟動停損
    
    # 守護者 6：倉位配置
    "HIGH_CONFIDENCE_ALLOCATION": 0.15, # 高信心：15% 資金
    "MEDIUM_CONFIDENCE_ALLOCATION": 0.08, # 中信心：8% 資金
}

print("✅ 參數設定完成")
print(f"\n當前設定：")
print(f"  - 停損線：{CONFIG['STOP_LOSS']:.0%}")
print(f"  - 獲利了結：{CONFIG['TAKE_PROFIT']:.0%}")
print(f"  - 流動性門檻：{CONFIG['MIN_TURNOVER']/1e6:.0f}M TWD")

## 🔧 第二步：載入主程式

In [ ]:
import random
from datetime import datetime, timedelta

# ==================== 📦 Mock Data 生成器 ====================
# 因為無法連接真實 API，所以用假資料模擬

def generate_market_data(scenario="bull"):
    """生成大盤資料"""
    if scenario == "bull":
        return {
            "index_name": "台灣加權指數",
            "current_price": 17500,
            "ma60": 17200,              # 季線
            "limit_down_count": 35,     # 跌停股票數
            "date": "2025-12-02"
        }
    else:  # bear
        return {
            "index_name": "台灣加權指數",
            "current_price": 16800,
            "ma60": 17200,
            "limit_down_count": 120,    # 恐慌！
            "date": "2025-12-02"
        }

def generate_mock_stocks(market_trend="bull"):
    """生成模擬股票（包含各種測試案例）"""
    
    stocks = []
    
    # === 測試案例 1：完美買點（台積電） ===
    stocks.append({
        "ticker": "2330",
        "name": "台積電",
        "price": 580,
        "ma20": 565,
        "ma60": 550,
        "ma120": 520,
        
        # 流動性資料
        "avg_volume_5d": 25000,                # 5 日均量（張）
        "avg_turnover_5d": 145_000_000,        # 5 日均成交金額（台幣）
        "today_volume": 28000,
        
        # 三大法人籌碼
        "chips": {
            "foreign": {                        # 外資
                "buy_days": 5,                  # 連續買超 5 天
                "today_amount": 150_000_000,    # 今日買超金額
                "today_ratio": 0.08             # 佔成交比 8%
            },
            "trust": {                          # 投信
                "buy_days": 4,
                "today_amount": 50_000_000,
                "today_ratio": 0.03
            },
            "dealer": {                         # 自營商
                "buy_days": 1,
                "today_amount": 10_000_000,
                "today_ratio": 0.01
            }
        },
        "total_turnover_today": 1_800_000_000
    })
    
    # === 測試案例 2：流動性陷阱 ===
    stocks.append({
        "ticker": "9999",
        "name": "殭屍股",
        "price": 15,
        "ma20": 14.5,
        "ma60": 14,
        "ma120": 13,
        "avg_volume_5d": 200,
        "avg_turnover_5d": 3_000_000,          # 只有 300 萬！太少了
        "today_volume": 180,
        "chips": {
            "foreign": {"buy_days": 2, "today_amount": 500_000, "today_ratio": 0.15},
            "trust": {"buy_days": 0, "today_amount": -100_000, "today_ratio": -0.03},
            "dealer": {"buy_days": 0, "today_amount": 50_000, "today_ratio": 0.01}
        },
        "total_turnover_today": 3_500_000
    })
    
    # === 測試案例 3：技術過熱 ===
    stocks.append({
        "ticker": "3324",
        "name": "雙鴻",
        "price": 720,
        "ma20": 650,
        "ma60": 520,                           # 乖離率 = (720-520)/520 = 38%！
        "ma120": 480,
        "avg_volume_5d": 8000,
        "avg_turnover_5d": 80_000_000,
        "today_volume": 9000,
        "chips": {
            "foreign": {"buy_days": 3, "today_amount": 20_000_000, "today_ratio": 0.06},
            "trust": {"buy_days": 2, "today_amount": 10_000_000, "today_ratio": 0.03},
            "dealer": {"buy_days": 0, "today_amount": -5_000_000, "today_ratio": -0.01}
        },
        "total_turnover_today": 350_000_000
    })
    
    # === 測試案例 4：籌碼背離（法人賣超） ===
    stocks.append({
        "ticker": "2603",
        "name": "長榮",
        "price": 150,
        "ma20": 148,
        "ma60": 145,
        "ma120": 140,
        "avg_volume_5d": 30000,
        "avg_turnover_5d": 200_000_000,
        "today_volume": 32000,
        "chips": {
            "foreign": {"buy_days": 0, "today_amount": -50_000_000, "today_ratio": -0.05},  # 外資賣超！
            "trust": {"buy_days": 0, "today_amount": -20_000_000, "today_ratio": -0.02},    # 投信也賣！
            "dealer": {"buy_days": 2, "today_amount": 5_000_000, "today_ratio": 0.01}
        },
        "total_turnover_today": 1_000_000_000
    })
    
    # === 測試案例 5：爆量警示 ===
    stocks.append({
        "ticker": "2454",
        "name": "聯發科",
        "price": 1020,
        "ma20": 1000,
        "ma60": 980,
        "ma120": 950,
        "avg_volume_5d": 5000,
        "avg_turnover_5d": 51_000_000,
        "today_volume": 28000,                 # 量比 = 28000/5000 = 5.6 倍！
        "chips": {
            "foreign": {"buy_days": 1, "today_amount": 30_000_000, "today_ratio": 0.04},
            "trust": {"buy_days": 0, "today_amount": -10_000_000, "today_ratio": -0.01},
            "dealer": {"buy_days": 0, "today_amount": 2_000_000, "today_ratio": 0.00}
        },
        "total_turnover_today": 750_000_000
    })
    
    return stocks

def generate_portfolio():
    """生成模擬持倉（測試出場策略）"""
    return [
        # 測試案例 1：獲利續抱
        {
            "ticker": "2317",
            "name": "鴻海",
            "cost_price": 100,
            "current_price": 115,
            "buy_date": "2025-11-10",
            "holding_days": 21,
            "max_price_since_buy": 120,
            "ma20": 110
        },
        # 測試案例 2：硬停損
        {
            "ticker": "2603",
            "name": "長榮",
            "cost_price": 100,
            "current_price": 91,            # 虧損 -9%
            "buy_date": "2025-11-25",
            "holding_days": 6,
            "max_price_since_buy": 102,
            "ma20": 95
        },
        # 測試案例 3：移動停利
        {
            "ticker": "2330",
            "name": "台積電",
            "cost_price": 500,
            "current_price": 560,
            "buy_date": "2025-10-15",
            "holding_days": 47,
            "max_price_since_buy": 630,     # 曾經漲到 630，現在 560
            "ma20": 580
        },
        # 測試案例 4：獲利了結
        {
            "ticker": "3711",
            "name": "日月光",
            "cost_price": 100,
            "current_price": 135,           # 獲利 +35%
            "buy_date": "2025-09-01",
            "holding_days": 91,
            "max_price_since_buy": 138,
            "ma20": 130
        },
    ]

print("✅ Mock Data 生成器載入完成")

## 🛡️ 第三步：六大守護者邏輯

In [ ]:
# ==================== 守護者 1：市場熔斷 ====================
def guardian_1_market_check(market_data, config):
    """
    判斷大盤是否安全
    - 檢查加權指數是否跌破季線（MA60）
    - 檢查跌停股票數量
    """
    current = market_data['current_price']
    ma60 = market_data['ma60']
    limit_down = market_data['limit_down_count']
    
    # 條件 1：大盤跌破季線
    below_ma60 = current < ma60
    
    # 條件 2：跌停家數過多（恐慌）
    panic = limit_down > config['MARKET_LIMIT_DOWN_THRESHOLD']
    
    if below_ma60 or panic:
        return {
            "status": "DANGER",
            "action": "停止買入",
            "reason": f"大盤 {current} < 季線 {ma60}" if below_ma60 else f"跌停 {limit_down} 支"
        }
    
    return {
        "status": "SAFE",
        "action": "允許買入",
        "reason": "市場正常"
    }

# ==================== 守護者 2：流動性檢查 ====================
def guardian_2_liquidity(stock, config):
    """
    檢查流動性（台股特色：用成交金額）
    - 5 日平均成交金額要 > 5000 萬
    - 偵測爆量（可能是出貨訊號）
    """
    turnover = stock['avg_turnover_5d']
    volume_ratio = stock['today_volume'] / stock['avg_volume_5d']
    
    # 主要條件：成交金額夠大
    if turnover < config['MIN_TURNOVER']:
        return {
            "pass": False,
            "reason": f"流動性不足（日均 {turnover/1e6:.1f}M < {config['MIN_TURNOVER']/1e6:.0f}M）"
        }
    
    # 輔助檢查：爆量警示
    if volume_ratio > config['VOLUME_SPIKE_RATIO']:
        return {
            "pass": True,
            "warning": f"⚠️ 異常爆量（量比 {volume_ratio:.1f}x）"
        }
    
    return {
        "pass": True,
        "reason": "流動性充足"
    }

# ==================== 守護者 3：籌碼共識 ====================
def guardian_3_chips(stock, config):
    """
    台股核心：三大法人籌碼分析
    - 外資 + 投信買超 = 強力訊號
    - 雙賣超 = 避開
    """
    foreign = stock['chips']['foreign']
    trust = stock['chips']['trust']
    dealer = stock['chips']['dealer']
    
    score = 0
    reasons = []
    
    # === 加分條件 ===
    
    # 1. 外資強力買超（連續 3 天 + 佔比 > 5%）
    if foreign['buy_days'] >= config['CONSECUTIVE_BUY_DAYS'] and foreign['today_ratio'] > config['FOREIGN_BUY_RATIO']:
        score += 2
        reasons.append(f"外資連{foreign['buy_days']}日買超（佔比{foreign['today_ratio']:.1%}）")
    
    # 2. 投信強力買超
    if trust['buy_days'] >= config['CONSECUTIVE_BUY_DAYS'] and trust['today_ratio'] > config['TRUST_BUY_RATIO']:
        score += 2
        reasons.append(f"投信連{trust['buy_days']}日買超（佔比{trust['today_ratio']:.1%}）")
    
    # 3. 三大法人同步買超
    if foreign['today_ratio'] > 0 and trust['today_ratio'] > 0 and dealer['today_ratio'] > 0:
        score += 1
        reasons.append("三大法人同步買超")
    
    # === 扣分條件 ===
    
    # 4. 外資投信雙賣超（警示）
    if foreign['today_ratio'] < -0.03 and trust['today_ratio'] < -0.02:
        score -= 3
        reasons.append("⚠️ 外資投信雙賣超")
    
    # 評級
    if score >= 3:
        level = "STRONG"
    elif score > 0:
        level = "MODERATE"
    elif score == 0:
        level = "NEUTRAL"
    else:
        level = "AVOID"
    
    return {
        "score": score,
        "level": level,
        "reasons": reasons
    }

# ==================== 守護者 4：技術面檢查 ====================
def guardian_4_technical(stock, config):
    """
    檢查股價是否過熱（乖離率）
    - 計算與 MA60 的乖離
    - 多頭市場允許較高乖離
    """
    price = stock['price']
    ma20 = stock['ma20']
    ma60 = stock['ma60']
    ma120 = stock['ma120']
    
    # 計算乖離率（使用 MA60 作為基準）
    bias = (price - ma60) / ma60
    
    # 判斷多空格局（均線多頭排列）
    is_bullish = (ma20 > ma60 > ma120)
    
    # 根據市場格局調整門檻
    if is_bullish:
        threshold = config['BIAS_THRESHOLD_BULL']  # 30%
    else:
        threshold = config['BIAS_THRESHOLD_BEAR']  # 15%
    
    if bias > threshold:
        return {
            "pass": False,
            "reason": f"技術過熱（乖離 {bias:.1%} > 門檻 {threshold:.0%}）"
        }
    
    return {
        "pass": True,
        "bias": bias,
        "trend": "多頭" if is_bullish else "空頭/盤整"
    }

# ==================== 守護者 5：出場策略 ====================
def guardian_5_exit(position, market_status, config):
    """
    動態停損停利（五層檢查）
    優先順序：
    1. 大盤熔斷（最優先）
    2. 硬停損（-8%）
    3. 技術停損（破線且虧損）
    4. 移動停利（保住獲利）
    5. 獲利了結（+30%）
    """
    cost = position['cost_price']
    current = position['current_price']
    max_high = position['max_price_since_buy']
    holding_days = position['holding_days']
    ma20 = position['ma20']
    
    # 計算指標
    pnl_ratio = (current - cost) / cost
    drawdown_from_peak = (max_high - current) / max_high
    highest_profit = (max_high - cost) / cost
    
    # === 第 1 層：大盤熔斷 ===
    if market_status == "DANGER":
        return {
            "action": "SELL",
            "reason": "🚨 大盤熔斷，強制出場",
            "type": "MARKET_CRASH"
        }
    
    # === 第 2 層：硬停損 ===
    if holding_days >= config['HOLDING_DAYS_MIN'] and pnl_ratio <= -config['STOP_LOSS']:
        return {
            "action": "SELL",
            "reason": f"✂️ 硬停損 -{config['STOP_LOSS']:.0%}（目前 {pnl_ratio:.1%}）",
            "type": "STOP_LOSS"
        }
    
    # === 第 3 層：技術停損 ===
    if current < ma20 and pnl_ratio < 0:
        return {
            "action": "SELL",
            "reason": f"📉 跌破月線且虧損（{pnl_ratio:.1%}）",
            "type": "TECH_BREAK"
        }
    
    # === 第 4 層：移動停利 ===
    if highest_profit > 0.10 and drawdown_from_peak > config['TRAILING_STOP']:
        return {
            "action": "SELL",
            "reason": f"📊 移動停利（高點回落 {drawdown_from_peak:.1%}）",
            "type": "TRAILING_STOP"
        }
    
    # === 第 5 層：獲利了結 ===
    if pnl_ratio >= config['TAKE_PROFIT']:
        return {
            "action": "SELL",
            "reason": f"🎯 獲利達標 +{config['TAKE_PROFIT']:.0%}（目前 {pnl_ratio:.1%}）",
            "type": "TAKE_PROFIT"
        }
    
    # === 續抱 ===
    return {
        "action": "HOLD",
        "reason": f"持有中（損益 {pnl_ratio:.1%}）",
        "type": "HOLD"
    }

# ==================== 守護者 6：倉位配置 ====================
def guardian_6_position(chips_score, config):
    """
    根據籌碼評分決定買入比例
    - 評分 >= 3：15% 資金
    - 評分 > 0：8% 資金
    - 評分 <= 0：不交易
    """
    if chips_score >= 3:
        return {
            "allocation": config['HIGH_CONFIDENCE_ALLOCATION'],
            "confidence": "HIGH",
            "description": "🔥 強力買入"
        }
    elif chips_score > 0:
        return {
            "allocation": config['MEDIUM_CONFIDENCE_ALLOCATION'],
            "confidence": "MEDIUM",
            "description": "⚡ 適度買入"
        }
    else:
        return {
            "allocation": 0,
            "confidence": "NONE",
            "description": "🚫 不交易"
        }

print("✅ 六大守護者載入完成")

## 🚀 第四步：主分析引擎

In [ ]:
def analyze_stock(stock, config):
    """分析單一股票（通過守護者 2-4-3-6）"""
    
    # 守護者 2：流動性
    liquidity = guardian_2_liquidity(stock, config)
    if not liquidity['pass']:
        return {
            "ticker": stock['ticker'],
            "name": stock['name'],
            "result": "FAIL",
            "reason": liquidity['reason']
        }
    
    # 守護者 4：技術面
    technical = guardian_4_technical(stock, config)
    if not technical['pass']:
        return {
            "ticker": stock['ticker'],
            "name": stock['name'],
            "result": "FAIL",
            "reason": technical['reason']
        }
    
    # 守護者 3：籌碼
    chips = guardian_3_chips(stock, config)
    
    # 守護者 6：倉位
    position = guardian_6_position(chips['score'], config)
    
    return {
        "ticker": stock['ticker'],
        "name": stock['name'],
        "price": stock['price'],
        "result": "PASS",
        "liquidity": liquidity,
        "technical": technical,
        "chips": chips,
        "position": position
    }

def run_analysis(market_scenario="bull"):
    """執行完整分析流程"""
    print(f"\n{'='*60}")
    print(f"📊 Manpan 情報網 v4.1 - 台股完整版")
    print(f"{'='*60}\n")
    
    # 1. 生成資料
    market = generate_market_data(market_scenario)
    stocks = generate_mock_stocks(market_scenario)
    portfolio = generate_portfolio()
    
    # 2. 守護者 1：市場檢查
    market_check = guardian_1_market_check(market, CONFIG)
    
    print(f"🌍 市場狀態檢查 (守護者 1)")
    print(f"{'─'*60}")
    status_icon = "🟢" if market_check['status'] == 'SAFE' else "🔴"
    print(f"狀態：{status_icon} {market_check['status']}")
    print(f"指數：{market['index_name']} {market['current_price']:,} 點")
    print(f"季線：{market['ma60']:,} 點")
    print(f"跌停：{market['limit_down_count']} 支")
    print(f"結論：{market_check['action']} ({market_check['reason']})")
    
    # 3. 分析候選股票
    print(f"\n{'='*60}")
    print(f"🔍 候選股票分析 (守護者 2-4-3-6)")
    print(f"{'─'*60}\n")
    
    results = []
    if market_check['status'] == 'SAFE':
        for stock in stocks:
            result = analyze_stock(stock, CONFIG)
            results.append(result)
            
            print(f"[{result['ticker']} {result['name']}]")
            print(f"{'─'*40}")
            
            if result['result'] == 'FAIL':
                print(f"🚫 淘汰：{result['reason']}\n")
            else:
                # 流動性
                liq = result['liquidity']
                if 'warning' in liq:
                    print(f"⚠️ 流動性：{liq['warning']}")
                else:
                    print(f"✅ 流動性：{liq['reason']}")
                
                # 技術面
                tech = result['technical']
                print(f"✅ 技術面：乖離 {tech['bias']:.1%}（{tech['trend']}）")
                
                # 籌碼面
                chips = result['chips']
                chip_icon = "🔥" if chips['level'] == 'STRONG' else "⚡" if chips['level'] == 'MODERATE' else "⚠️"
                print(f"{chip_icon} 籌碼面：{chips['level']} (評分 {chips['score']})")
                for reason in chips['reasons']:
                    print(f"   └─ {reason}")
                
                # 倉位建議
                pos = result['position']
                print(f"💰 建議倉位：{pos['allocation']:.0%} ({pos['description']})\n")
    else:
        print("⚠️ 市場熔斷，停止分析候選股票\n")
    
    # 4. 持倉健檢
    print(f"\n{'='*60}")
    print(f"📦 持倉健檢 (守護者 5)")
    print(f"{'─'*60}\n")
    
    for pos in portfolio:
        exit_signal = guardian_5_exit(pos, market_check['status'], CONFIG)
        pnl = (pos['current_price'] - pos['cost_price']) / pos['cost_price']
        
        print(f"[{pos['ticker']} {pos['name']}]")
        print(f"成本：{pos['cost_price']} → 現價：{pos['current_price']} ({pnl:+.1%})")
        
        if exit_signal['action'] == 'SELL':
            print(f"動作：🔔 {exit_signal['action']}")
            print(f"原因：{exit_signal['reason']}\n")
        else:
            print(f"動作：✅ {exit_signal['action']}")
            print(f"說明：{exit_signal['reason']}\n")
    
    # 5. 統計摘要
    passed = [r for r in results if r['result'] == 'PASS' and r['position']['allocation'] > 0]
    failed = [r for r in results if r['result'] == 'FAIL']
    
    print(f"\n{'='*60}")
    print(f"📊 執行摘要")
    print(f"{'─'*60}")
    print(f"✅ 通過篩選：{len(passed)} 支")
    print(f"🚫 淘汰：{len(failed)} 支")
    print(f"\n建議配置：")
    for r in passed:
        print(f"   - {r['ticker']} {r['name']}：{r['position']['allocation']:.0%}")
    print(f"{'='*60}\n")

print("✅ 分析引擎載入完成")

## 🎮 第五步：執行分析

In [ ]:
# 執行分析（多頭市場）
run_analysis(market_scenario="bull")

## 🐻 第六步：測試空頭市場

In [ ]:
# 執行分析（空頭市場）
run_analysis(market_scenario="bear")

---

## 📚 學習重點

### 台股投資核心概念

1. **三大法人**
   - 外資：外國機構投資人（資金最大）
   - 投信：本土基金公司（操作靈活）
   - 自營商：券商自己的錢（避險為主）

2. **籌碼面指標**
   - 買超：法人買進 > 賣出
   - 連續買超：3 天以上更有意義
   - 買超佔比：買超金額 / 當日成交額

3. **技術面指標**
   - 乖離率：股價偏離均線的程度
   - MA20/60/120：月線/季線/半年線
   - 多頭排列：短均線 > 長均線

4. **流動性**
   - 為什麼用「成交金額」而非「成交量」？
   - 因為高價股（如台積電 600 元）和低價股（如 10 元股）成交量沒有可比性
   - 統一用「日均成交 5000 萬」作為門檻

5. **風險管理**
   - 停損：避免大虧（-8%）
   - 停利：鎖住獲利（從高點回落 10%）
   - 大盤熔斷：系統性風險優先

---

## 🔧 進階調整（你可以試試看）

修改 `CONFIG` 的參數，看結果有什麼變化：

```python
# 更激進的設定
CONFIG['STOP_LOSS'] = 0.05  # 改成 -5% 停損（更快停損）
CONFIG['TAKE_PROFIT'] = 0.20  # 改成 +20% 獲利了結（更早賣）

# 更保守的設定
CONFIG['MIN_TURNOVER'] = 100_000_000  # 改成 1 億（只買大型股）
CONFIG['FOREIGN_BUY_RATIO'] = 0.10  # 改成 10%（要求更強的法人買盤）
```

---

## ✅ 完成！

現在你有一個：
- ✅ 完整的台股分析系統
- ✅ 六大守護者邏輯
- ✅ 詳細的中文說明
- ✅ 可以自己調整參數

**學習方式：**
1. 先跑一次看結果
2. 讀每個函數的註解
3. 改參數看變化
4. 查不懂的名詞（外資、乖離率等）